In [1]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm

# Set fixed random number seed
torch.manual_seed(777)

# Load datasets
dataset = pd.read_csv('./practice_data.csv')


# Split target and variables
X = dataset.drop(['DATETIME', 'QGEN'], axis=1).to_numpy()
Y = dataset['QGEN'].to_numpy().reshape(-1,1)


# Split train and test data
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.1, shuffle=False, random_state=777)



# Normalization
def normalization(numpy_array):
    df = pd.DataFrame(numpy_array).copy()
    
    # Column index is number of variables
    for column_index in range(len(df.columns)):
        df[column_index] = (df[column_index] - df[column_index].min()) / (df[column_index].max() - df[column_index].min())
    
    # Convert dataframe to numpy
    normalized_df = df.to_numpy()
    
    return normalized_df

x_train_norm = normalization(x_train)
y_train_norm = normalization(y_train)

x_test_norm = normalization(x_test)
y_test_norm = normalization(y_test)



# Convert numpy to tensor
class TensorData(Dataset): # torch의 Dataset 클래스 상속
    def __init__(self, x_data, y_data):
        self.x_data = torch.FloatTensor(x_data)
        self.y_data = torch.FloatTensor(y_data)
        self.len = self.y_data.shape[0]

    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index] 

    def __len__(self):
        return self.len
    
    
# Iteration 객체 반환하는 데이터로더
trainsets = TensorData(x_train_norm, y_train)
trainloader = torch.utils.data.DataLoader(trainsets, batch_size=64, shuffle=True)

testsets = TensorData(x_test_norm, y_test)
testloader = torch.utils.data.DataLoader(testsets, batch_size=16, shuffle=False)



# Define a Artifical Neural Network
class NeuralNetwork(nn.Module):
    def __init__(self):
        """모델 연산 정의"""
        super(NeuralNetwork, self).__init__()
        self.fc_input = nn.Linear(5, 32, bias=True, device=None)        # input_node=5, output_node=32
        self.fc_hidden1 = nn.Linear(32, 64, bias=True, device=None)     # input_node=32, output_node=64
        self.fc_hidden2 = nn.Linear(64, 64, bias=True, device=None)     # input_node=64, output_node=64
        self.fc_hidden3 = nn.Linear(64, 32, bias=True, device=None)     # input_node=64, output_node=32
        self.fc_out = nn.Linear(32, 1, bias=True, device=None)          # input_node=32, output_node=1
        self.dropout = nn.Dropout(0.2)                                  # 일정 비율로 랜덤하게 Node 제거
    
    
    def forward(self, x):
        """모델 연산 순서 정의"""
        x = F.relu(self.fc_input(x))                                    # Linear 계산 후 활성화 함수 적용
        x = F.relu(self.fc_hidden1(x))
        x = F.relu(self.fc_hidden2(x))
#         x = self.dropout(F.relu(self.fc_hidden2(x)))
        x = F.relu(self.fc_hidden3(x))
        out = F.relu(self.fc_out(x))
        
        return out


# Create ANN Instance
model = NeuralNetwork()
print(model)


# Define a Loss function and optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.0003)
loss_func = torch.nn.MSELoss()



# Training
loss_epoch = []
for i in tqdm(range(1000)): # 1000 epoch 
    
    batch_loss = 0.0 # 한 에폭이 돌 때 배치마다 loss가 나온다. 즉 한번 학습할 때 배치마다 loss가 나오니 MSE를 구하기 위해서 초기화
    
    for idx, batch in enumerate(trainloader):

        inputs, labels = batch                 # 배치 데이터
        
        optimizer.zero_grad()                  # 옵티마이저 초기화 (역전파 전에 그래디언트 초기화)
        outputs = model(inputs)                # 모델에 입력값을 넣어 예측값 산출
        loss = loss_func(outputs, labels)      # compute loss
        loss.backward()                        # compute gradients (dloss/dx)
        optimizer.step()                       # 역전파를 진행하고 가중치를 업데이트
        
        batch_loss += loss.item()
        
    loss_epoch.append(batch_loss / trainloader.dataset.__len__())
    
plt.plot(loss_epoch)
plt.title('Loss(MSE)')
plt.xlabel('Epoch')
plt.show()

print(loss_epoch)

ModuleNotFoundError: No module named 'torch'